### Name: Parth Kothari
### roll no. : msc2303121012
### Course: MSc

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib tk

# $\circ$ Total Variation
This type of schemes are used to measure oscillations in a numerical solution for a given quantity $\textbf{'q'}$ . The formula for oscillation measurement is :
$$T V (q) = \sum_{i=1}^{N} |q_{i} − q_{i−1}|  $$
where i = 1 is the left most cell and N is the right most cell of the grid.

### $\circ$ Monotonically Increasing or decreasing Function (No Oscillations):
1) If the function q is monotonic that is either it is $\textbf{decreasing or increasing}$ at a constant rate,  $ TV = constant $ . IF the function is constant, then $TV = 0$. This means there is no local maxima or minima in the numerical grid solution.
2) AS long as the function is monotic, TV is constant and there are no oscillations in the scheme.
### $\circ$ Non - monotonous function (Oscillations):
1) In case of non monotonous function, there will be sharp increase in the values of the quantity leading to local maxima or minima in the function, making the $TV(q)$ increase with each step.
2) Therefore $\textbf{increasing TV values} $ indicate towards the development of oscillations at the sharp descent or ascent of the function. 

# $\circ$ TVD scheme
A numerical scheme is said to be total variation diminishing (TVD) if:
$$T V (q^{n+1}) ≤ T V (q^{n}) $$
where 'q' is the quantity we are numerically solving, 'n' being the time step.
In simple words this type of schemes would make the ascent or descent of the function smooth as it is keeping the TV values constant or less than the previous values, minimizing the oscillations this way. There will the loss of some solution due to this but the oscillations would be removed. 



# Slope Limiters
Slope Limiters are schemes which limit the ascent or descent of the quantity to some value for different scheme types. It basically prevents overshooting at sharp edges by limiting the slope of the line that joins the center of the grid. This is similiar to adding some error to the solution to make it stable.




# Flux limiters
Flux limiters are similar to slope limiters but instead of limiting the slope, It limits the flow of the quantity itself (flux). This type of schemes help when there is conservative equarions are present where we have to conserve the quantity.
The equation for the limiter:
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}u_{i-\frac{1}{2}} \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right] + \frac{1}{2}|u_{i-\frac{1}{2}}| \left(1 - \left |\frac{u_{i-\frac{1}{2}}\Delta t}{\Delta x}\right|  \right) \Phi(r^n_{i-\frac{1}{2}}) (q^n_i - q^n_{i-1})
  $$

$$f^{n+\frac{1}{2}}_{i+\frac{1}{2}} = \frac{1}{2}u_{i+\frac{1}{2}} \left[ (1 + \theta_{i+\frac{1}{2}})q^n_i + (1 - \theta_{i+\frac{1}{2}})q^n_{i+1} \right] + \frac{1}{2}|u_{i+\frac{1}{2}}| \left(1 - \left |\frac{u_{i+\frac{1}{2}}\Delta t}{\Delta x}\right|  \right) \Phi(r^n_{i+\frac{1}{2}}) (q^n_{i+1} - q^n_i)
 $$

For our case the velocity is constant therefore the above formula reduces down to :
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) \Phi(r^n_{i-\frac{1}{2}}) (q^n_i - q^n_{i-1})
 $$

$$f^{n+\frac{1}{2}}_{i+\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i+\frac{1}{2}})q^n_i + (1 - \theta_{i+\frac{1}{2}})q^n_{i+1} \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) \Phi(r^n_{i+\frac{1}{2}}) (q^n_{i+1} - q^n_i)
 $$

## Donner cell
For donner cell , $\phi  = 0 $, The formula reduces to:
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right]$$

$$f^{n+\frac{1}{2}}_{i+\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i+\frac{1}{2}})q^n_i + (1 - \theta_{i+\frac{1}{2}})q^n_{i+1} \right]$$

where 
$$ \theta_{i-\frac{1}{2}} \equiv \theta(u_{i-\frac{1}{2}}) = 
\begin{cases} 
+1 & \text{for } \lambda \geq 0 \\
-1 & \text{for } \lambda \leq 0 
\end{cases} $$ 

Same for i+1/2 case



In [23]:
steps = 200  # Initializing the grid
x = np.linspace(-10, 10, steps) 
t = np.linspace(0,25,1000)
dx = x[1] - x[0]
dt = t[1] - t[0]

pulse_height = 3.0
pulse_width = 8.0
u_0 = np.where((np.abs(x) <= pulse_width / 2), pulse_height, 0)   #Defining the rectangular pulse
lamb = 3.0  #speed of advection

fig, ax = plt.subplots()  #animation plot code block
line, = ax.plot(x, u_0,color = 'Black')
ax.set_xlabel("x")
ax.set_ylabel("advecting quantity")    
ax.set_title(f"Donor cell, $\lambda $ = {lamb} ")

u_values_for_step = u_0.copy()  #The values that will be updated after each time step


def update(frame):
    u = u_values_for_step.copy()  # using previous values as copies to update the new ones for each frame 
    for i in range(len(x)):
        if lamb > 0:
         theta = 1.0
        else:
           theta = -1.0

        f_minus_half =  (lamb/2) * ( (1 + theta) * u[(i - 1) % steps] + (1-theta) * u[(i) % steps] )
        f_plus_half = (lamb/2) * ( (1 + theta) * u[(i) % steps] + (1-theta) * u[(i+1) % steps] )

        u_values_for_step[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)
    line.set_ydata(u_values_for_step)

    return line,

animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)
plt.show()


## Lax Wenderoff
For lax Wenderoff $\phi  = 1.0$. The formula becomes:
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) (1) (q^n_i - q^n_{i-1}) $$
$$f^{n+\frac{1}{2}}_{i+\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i+\frac{1}{2}})q^n_i + (1 - \theta_{i+\frac{1}{2}})q^n_{i+1} \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) (1) (q^n_{i+1} - q^n_i) $$

In [5]:
steps = 200
x = np.linspace(-10, 10, steps) 
t = np.linspace(0,25,1000)
dx = x[1] - x[0]
dt = t[1] - t[0]

pulse_height = 3.0
pulse_width = 8.0
u_0 = np.where((np.abs(x) <= pulse_width / 2), pulse_height, 0) 
lamb = 3.0

fig, ax = plt.subplots()
line, = ax.plot(x, u_0,color = 'red')
ax.set_xlabel("x")
ax.set_ylabel("advecting quantity")    
ax.set_title(f"Lax wenderoff, $\lambda $ = {lamb} ")

u_values_for_step = u_0.copy()


def update(frame):
    u = u_values_for_step.copy()
    for i in range(len(x)):
        
        if lamb > 0:
         theta = 1.0
        else:
           theta = -1.0
        
        f_minus_half =  (lamb/2) * ( (1 + theta) * u[(i - 1) % steps] + (1-theta) * u[(i) % steps] ) + (abs(lamb)/2) * (1 - abs((lamb * dt / dx))) * (u[i % steps] - u[(i-1)% steps])
        f_plus_half = (lamb/2) * ( (1 + theta) * u[(i) % steps] + (1-theta) * u[(i+1) % steps] ) + (abs(lamb)/2) * (1 - abs((lamb * dt / dx))) * (u[(i+1) % steps] - u[(i)% steps])

        u_values_for_step[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)
    line.set_ydata(u_values_for_step)

    return line,

animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)
plt.show()


## Beam Warming
$\phi = r$, formula:
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) r^n_{i-\frac{1}{2}} (q^n_i - q^n_{i-1})
 $$
$$f^{n+\frac{1}{2}}_{i+\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i+\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i+\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) r^n_{i+\frac{1}{2}} (q^n_{i+1} - q^n_{i})
 $$

where
$$ r_{i-\frac{1}{2}} = 
\begin{cases}
    \frac{q^n_{i-1} - q^n_{i-2}}{q^n_{i} - q^n_{i-1}} & \text{for } \lambda \geq 0 \\
    \frac{q^n_{i+1} - q^n_i}{q^n_{i} - q^n_{i-1}} & \text{for } \lambda \leq 0
\end{cases} $$

$$ r_{i+\frac{1}{2}} = 
\begin{cases}
    \frac{q^n_{i} - q^n_{i-1}}{q^n_{i+1} - q^n_{i}} & \text{for } \lambda \geq 0 \\
    \frac{q^n_{i+2} - q^n_{i+1}}{q^n_{i+1} - q^n_{i}} & \text{for } \lambda \leq 0
\end{cases} $$



In [ ]:
# # Accidental good scheme
# steps = 200
# x = np.linspace(-10, 10, steps) 
# t = np.linspace(0,25,1000)
# dx = x[1] - x[0]
# dt = t[1] - t[0]

# pulse_height = 3.0
# pulse_width = 8.0
# u_0 = np.where((np.abs(x) <= pulse_width / 2), pulse_height, 0) 
# lamb = 3.0

# fig, ax = plt.subplots()
# line, = ax.plot(x, u_0,color = 'Magenta')
# ax.set_xlabel("x")
# ax.set_ylabel("advection")    
# ax.set_title(f"Beam warming, $\lambda $ = {lamb} ")

# u_values_for_step = u_0.copy()


# def update(frame):
#     u = u_values_for_step.copy()
#     for i in range(len(x)):
        
#         theta = 1.0
#         r_minus_half_denominator = (u[(i)%steps] - u[(i-2)%steps]) # Accidentally, took i-2 here, and the scheme got stable
#         r_minus_half = (u[(i-1)%steps] - u[(i-2)%steps]) / r_minus_half_denominator if r_minus_half_denominator!=0 else 0
        
#         r_plus_half_denominator = (u[(i+1)%steps] - u[(i)%steps])
#         r_plus_half = (u[(i)%steps] - u[(i-1)%steps]) / r_plus_half_denominator if r_plus_half_denominator !=0 else 0
        
        
#         f_minus_half =  (lamb/2) * ( (1 + theta) * u[(i - 1) % steps] + (1-theta) * u[(i) % steps] ) + (lamb/2) * (1 - (lamb * dt / dx)) * (u[i % steps] - u[(i-1)% steps]) * r_minus_half
#         f_plus_half = (lamb/2) * ( (1 + theta) * u[(i) % steps] + (1-theta) * u[(i+1) % steps] ) + (lamb/2) * (1 - (lamb * dt / dx)) * (u[(i+1) % steps] - u[(i)% steps]) * r_plus_half

#         u_values_for_step[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)
#     line.set_ydata(u_values_for_step)

#     return line,

# animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)
# plt.show()


In [6]:
# Actual Scheme
steps = 200
x = np.linspace(-10, 10, steps) 
t = np.linspace(0,25,1000)
dx = x[1] - x[0]
dt = t[1] - t[0]

pulse_height = 3.0
pulse_width = 8.0
u_0 = np.where((np.abs(x) <= pulse_width / 2), pulse_height, 0) 
lamb = 3.0

fig, ax = plt.subplots()
line, = ax.plot(x, u_0,color = 'Magenta')
ax.set_xlabel("x")
ax.set_ylabel("advecting quantity")    
ax.set_title(f"Beam warning, $\lambda $ = {lamb} ")

u_values_for_step = u_0.copy()


def update(frame):
    u = u_values_for_step.copy()
    for i in range(len(x)):
        
        if lamb > 0:
           theta = 1.0
           r_minus_half_denominator = (u[(i)%steps] - u[(i-1)%steps])  # two values as we have conditions for r as well.
           r_minus_half = (u[(i-1)%steps] - u[(i-2)%steps]) / r_minus_half_denominator if r_minus_half_denominator!=0 else 0
            
           r_plus_half_denominator = (u[(i+1)%steps] - u[(i)%steps])
           r_plus_half = (u[(i)%steps] - u[(i-1)%steps]) / r_plus_half_denominator if r_plus_half_denominator !=0 else 0
        
        else:
           theta = -1.0
           r_minus_half_denominator = (u[(i)%steps] - u[(i-1)%steps])  
           r_minus_half = (u[(i+1)%steps] - u[(i)%steps]) / r_minus_half_denominator if r_minus_half_denominator!=0 else 0
            
           r_plus_half_denominator = (u[(i+1)%steps] - u[(i)%steps])
           r_plus_half = (u[(i+2)%steps] - u[(i+1)%steps]) / r_plus_half_denominator if r_plus_half_denominator !=0 else 0
        
        
        
        f_minus_half =  (lamb/2) * ( (1 + theta) * u[(i - 1) % steps] + (1-theta) * u[(i) % steps] ) + abs((lamb/2)) * (1 - abs((lamb * dt / dx)) ) * (u[i % steps] - u[(i-1)% steps]) * r_minus_half
        f_plus_half = (lamb/2) * ( (1 + theta) * u[(i) % steps] + (1-theta) * u[(i+1) % steps] ) + abs((lamb/2)) * (1 - abs((lamb * dt / dx))) * (u[(i+1) % steps] - u[(i)% steps]) * r_plus_half

        u_values_for_step[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)
    line.set_ydata(u_values_for_step)

    return line,

animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)
plt.show()


## Fromm 
$\phi = \frac{1}{2} (1+r) $. The formula changes to:
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) \frac{1}{2}(1 + r^n_{i-\frac{1}{2}}) (q^n_i - q^n_{i-1})
 $$
$$f^{n+\frac{1}{2}}_{i+\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i+\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i+\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) \frac{1}{2}(1 + r^n_{i+\frac{1}{2}}) (q^n_{i+1} - q^n_{i})
 $$

In [28]:

steps = 200
x = np.linspace(-10, 10, steps) 
t = np.linspace(0,25,1000)
dx = x[1] - x[0]
dt = t[1] - t[0]

pulse_height = 3.0
pulse_width = 8.0
u_0 = np.where((np.abs(x) <= pulse_width / 2), pulse_height, 0) 
lamb = -3.0

fig, ax = plt.subplots()
line, = ax.plot(x, u_0,color = 'green')
ax.set_xlabel("x")
ax.set_ylabel("advection")    
ax.set_title(f"Fromm, $\lambda $ = {lamb} ")

u_values_for_step = u_0.copy()


def update(frame):
    u = u_values_for_step.copy()
    for i in range(len(x)):
        
        if lamb > 0:
           theta = 1.0
           r_minus_half_denominator = (u[(i)%steps] - u[(i-1)%steps])  # two values as we have conditions for r as well.
           r_minus_half = (u[(i-1)%steps] - u[(i-2)%steps]) / r_minus_half_denominator if r_minus_half_denominator!=0 else 0
            
           r_plus_half_denominator = (u[(i+1)%steps] - u[(i)%steps])
           r_plus_half = (u[(i)%steps] - u[(i-1)%steps]) / r_plus_half_denominator if r_plus_half_denominator !=0 else 0
        
        else:
           theta = -1.0
           r_minus_half_denominator = (u[(i)%steps] - u[(i-1)%steps])  
           r_minus_half = (u[(i+1)%steps] - u[(i)%steps]) / r_minus_half_denominator if r_minus_half_denominator!=0 else 0
            
           r_plus_half_denominator = (u[(i+1)%steps] - u[(i)%steps])
           r_plus_half = (u[(i+2)%steps] - u[(i+1)%steps]) / r_plus_half_denominator if r_plus_half_denominator !=0 else 0
        
        
        f_minus_half =  (lamb/2) * ( (1 + theta) * u[(i - 1) % steps] + (1-theta) * u[(i) % steps] ) + abs((lamb/2)) * (1 - abs((lamb * dt / dx))) * (u[i % steps] - u[(i-1)% steps]
                        ) * ((1 + r_minus_half) / 2)
        f_plus_half = (lamb/2) * ( (1 + theta) * u[(i) % steps] + (1-theta) * u[(i+1) % steps] ) + abs((lamb/2)) * (1 - abs((lamb * dt / dx))) * (u[(i+1) % steps] - u[(i)% steps]
                        ) * ((1 + r_plus_half) / 2)

        u_values_for_step[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)
    line.set_ydata(u_values_for_step)

    return line,

animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)
plt.show()


## Van leer
$\phi =\frac{r + |r|}{1+|r|}  $ The formula changes to:
$$f^{n+\frac{1}{2}}_{i-\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i-\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i-\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) \frac{r_{i-1/2} + |r_{i-1/2}|}{1+|r_{i-1/2}|} (q^n_i - q^n_{i-1})
 $$
$$f^{n+\frac{1}{2}}_{i+\frac{1}{2}} = \frac{1}{2}\lambda \left[ (1 + \theta_{i+\frac{1}{2}})q^n_{i-1} + (1 - \theta_{i+\frac{1}{2}})q^n_i \right] + \frac{1}{2}|\lambda| \left(1 - \left |\frac{\lambda\Delta t}{\Delta x}\right|  \right) \frac{r_{i+1/2} + |r_{i+1/2}|}{1+|r_{i+1/2}|} (q^n_{i+1} - q^n_{i})
 $$

In [33]:

steps = 200
x = np.linspace(-10, 10, steps) 
t = np.linspace(0,25,1000)
dx = x[1] - x[0]
dt = t[1] - t[0]

pulse_height = 3.0
pulse_width = 8.0
u_0 = np.where((np.abs(x) <= pulse_width / 2), pulse_height, 0) 
lamb = -3.0

fig, ax = plt.subplots()
line, = ax.plot(x, u_0,color = 'orange')
ax.set_xlabel("x")
ax.set_ylabel("advection")    
ax.set_title(f"Van leer, $\lambda $ = {lamb} ")

u_values_for_step = u_0.copy()


def update(frame):
    u = u_values_for_step.copy()
    for i in range(len(x)):
        
        if lamb > 0:
           theta = 1.0
           r_minus_half_denominator = (u[(i)%steps] - u[(i-1)%steps])  # two values as we have conditions for r as well.
           r_minus_half = (u[(i-1)%steps] - u[(i-2)%steps]) / r_minus_half_denominator if r_minus_half_denominator!=0 else 0
            
           r_plus_half_denominator = (u[(i+1)%steps] - u[(i)%steps])
           r_plus_half = (u[(i)%steps] - u[(i-1)%steps]) / r_plus_half_denominator if r_plus_half_denominator !=0 else 0
        
        else:
           theta = -1.0
           r_minus_half_denominator = (u[(i)%steps] - u[(i-1)%steps])  
           r_minus_half = (u[(i+1)%steps] - u[(i)%steps]) / r_minus_half_denominator if r_minus_half_denominator!=0 else 0
            
           r_plus_half_denominator = (u[(i+1)%steps] - u[(i)%steps])
           r_plus_half = (u[(i+2)%steps] - u[(i+1)%steps]) / r_plus_half_denominator if r_plus_half_denominator !=0 else 0
        
        
        f_minus_half =  (lamb/2) * ( (1 + theta) * u[(i - 1) % steps] + (1-theta) * u[(i) % steps] ) + abs((lamb/2)) * (1 - abs((lamb * dt / dx))) * (u[i % steps] - u[(i-1)% steps]
                        ) * ( (r_minus_half + abs(r_minus_half)) / (1 + abs(r_minus_half)) )
        f_plus_half = (lamb/2) * ( (1 + theta) * u[(i) % steps] + (1-theta) * u[(i+1) % steps] ) + abs((lamb/2)) * (1 - abs((lamb * dt / dx))) * (u[(i+1) % steps] - u[(i)% steps]
                        ) * ( (r_plus_half + abs(r_plus_half)) / (1 + abs(r_plus_half)) )

        u_values_for_step[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)
    line.set_ydata(u_values_for_step)

    return line,

animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)
plt.show()


## Min mod
$\phi = minmod(1,r) $ 

In [34]:
# Actual Scheme
steps = 200
x = np.linspace(-10, 10, steps) 
t = np.linspace(0,25,1000)
dx = x[1] - x[0]
dt = t[1] - t[0]

pulse_height = 3.0
pulse_width = 8.0
u_0 = np.where((np.abs(x) <= pulse_width / 2), pulse_height, 0) 
lamb = -3.0

fig, ax = plt.subplots()
line, = ax.plot(x, u_0,color = 'red')
ax.set_xlabel("x")
ax.set_ylabel("advection")    
ax.set_title(f"Min Mod, $\lambda $ = {lamb} ")

u_values_for_step = u_0.copy()


def update(frame):
    u = u_values_for_step.copy()
    for i in range(len(x)):
        
        if lamb > 0:
           theta = 1.0
           r_minus_half_denominator = (u[(i)%steps] - u[(i-1)%steps])  # two values as we have conditions for r as well.
           r_minus_half = (u[(i-1)%steps] - u[(i-2)%steps]) / r_minus_half_denominator if r_minus_half_denominator!=0 else 0
            
           r_plus_half_denominator = (u[(i+1)%steps] - u[(i)%steps])
           r_plus_half = (u[(i)%steps] - u[(i-1)%steps]) / r_plus_half_denominator if r_plus_half_denominator !=0 else 0
        
        else:
           theta = -1.0
           r_minus_half_denominator = (u[(i)%steps] - u[(i-1)%steps])  
           r_minus_half = (u[(i+1)%steps] - u[(i)%steps]) / r_minus_half_denominator if r_minus_half_denominator!=0 else 0
            
           r_plus_half_denominator = (u[(i+1)%steps] - u[(i)%steps])
           r_plus_half = (u[(i+2)%steps] - u[(i+1)%steps]) / r_plus_half_denominator if r_plus_half_denominator !=0 else 0
        
        
        f_minus_half =  (lamb/2) * ( (1 + theta) * u[(i - 1) % steps] + (1-theta) * u[(i) % steps] ) + abs((lamb/2)) * (1 - abs((lamb * dt / dx))) * (u[i % steps] - u[(i-1)% steps]) * min(1,abs(r_minus_half))
        f_plus_half = (lamb/2) * ( (1 + theta) * u[(i) % steps] + (1-theta) * u[(i+1) % steps] ) + abs((lamb/2)) * (1 - abs((lamb * dt / dx))) * (u[(i+1) % steps] - u[(i)% steps]) * min(1,abs(r_plus_half))

        u_values_for_step[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)
    line.set_ydata(u_values_for_step)

    return line,

animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)
plt.show()


## Super bee
$\phi = max(0, min(1,2r), min(2,r)) $ 


In [37]:
# Actual Scheme
steps = 200
x = np.linspace(-10, 10, steps) 
t = np.linspace(0,25,1000)
dx = x[1] - x[0]
dt = t[1] - t[0]

pulse_height = 3.0
pulse_width = 8.0
u_0 = np.where((np.abs(x) <= pulse_width / 2), pulse_height, 0) 
lamb = -3.0

fig, ax = plt.subplots()
line, = ax.plot(x, u_0,color = 'tomato')
ax.set_xlabel("x")
ax.set_ylabel("advection")    
ax.set_title(f"Super bee, $\lambda $ = {lamb} ")

u_values_for_step = u_0.copy()


def update(frame):
    u = u_values_for_step.copy()
    for i in range(len(x)):
        
        if lamb > 0:
           theta = 1.0
           r_minus_half_denominator = (u[(i)%steps] - u[(i-1)%steps])  # two values as we have conditions for r as well.
           r_minus_half = (u[(i-1)%steps] - u[(i-2)%steps]) / r_minus_half_denominator if r_minus_half_denominator!=0 else 0
            
           r_plus_half_denominator = (u[(i+1)%steps] - u[(i)%steps])
           r_plus_half = (u[(i)%steps] - u[(i-1)%steps]) / r_plus_half_denominator if r_plus_half_denominator !=0 else 0
        
        else:
           theta = -1.0
           r_minus_half_denominator = (u[(i)%steps] - u[(i-1)%steps])  
           r_minus_half = (u[(i+1)%steps] - u[(i)%steps]) / r_minus_half_denominator if r_minus_half_denominator!=0 else 0
            
           r_plus_half_denominator = (u[(i+1)%steps] - u[(i)%steps])
           r_plus_half = (u[(i+2)%steps] - u[(i+1)%steps]) / r_plus_half_denominator if r_plus_half_denominator !=0 else 0
        
        
        f_minus_half =  (lamb/2) * ( (1 + theta) * u[(i - 1) % steps] + (1-theta) * u[(i) % steps] ) + abs((lamb/2)) * (1 - abs((lamb * dt / dx))) * (u[i % steps] - u[(i-1)% steps]) * max(0,min(1,2*r_minus_half),min(2,r_minus_half))
        f_plus_half = (lamb/2) * ( (1 + theta) * u[(i) % steps] + (1-theta) * u[(i+1) % steps] ) + abs((lamb/2)) * (1 - abs((lamb * dt / dx))) * (u[(i+1) % steps] - u[(i)% steps]) * max(0,min(1,2*r_plus_half),min(2,r_plus_half))

        u_values_for_step[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)
    line.set_ydata(u_values_for_step)

    return line,

animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)
plt.show()


## MC
$\phi = max(0, min((1 + r)/2, 2, 2r)) $

In [3]:
# Actual Scheme
steps = 200
x = np.linspace(-10, 10, steps) 
t = np.linspace(0,25,1000)
dx = x[1] - x[0]
dt = t[1] - t[0]

pulse_height = 3.0
pulse_width = 8.0
u_0 = np.where((np.abs(x) <= pulse_width / 2), pulse_height, 0) 
lamb = -3.0

fig, ax = plt.subplots()
line, = ax.plot(x, u_0,color = 'lawngreen')
ax.set_xlabel("x")
ax.set_ylabel("advection")    
ax.set_title(f"MC, $\lambda $ = {lamb} ")

u_values_for_step = u_0.copy()


def update(frame):
    u = u_values_for_step.copy()
    for i in range(len(x)):
        
        if lamb > 0:
           theta = 1.0
           r_minus_half_denominator = (u[(i)%steps] - u[(i-1)%steps])  # two values as we have conditions for r as well.
           r_minus_half = (u[(i-1)%steps] - u[(i-2)%steps]) / r_minus_half_denominator if r_minus_half_denominator!=0 else 0
            
           r_plus_half_denominator = (u[(i+1)%steps] - u[(i)%steps])
           r_plus_half = (u[(i)%steps] - u[(i-1)%steps]) / r_plus_half_denominator if r_plus_half_denominator !=0 else 0
        
        else:
           theta = -1.0
           r_minus_half_denominator = (u[(i)%steps] - u[(i-1)%steps])  
           r_minus_half = (u[(i+1)%steps] - u[(i)%steps]) / r_minus_half_denominator if r_minus_half_denominator!=0 else 0
            
           r_plus_half_denominator = (u[(i+1)%steps] - u[(i)%steps])
           r_plus_half = (u[(i+2)%steps] - u[(i+1)%steps]) / r_plus_half_denominator if r_plus_half_denominator !=0 else 0
        
        
        f_minus_half =  (lamb/2) * ( (1 + theta) * u[(i - 1) % steps] + (1-theta) * u[(i) % steps] ) + abs((lamb/2)) * (1 - abs((lamb * dt / dx))) * (u[i % steps] - u[(i-1)% steps]) * max(0,min((1 + r_minus_half)/2, 2, 2*r_minus_half))
        f_plus_half = (lamb/2) * ( (1 + theta) * u[(i) % steps] + (1-theta) * u[(i+1) % steps] ) + abs((lamb/2)) * (1 - abs((lamb * dt / dx))) * (u[(i+1) % steps] - u[(i)% steps]) * max(0,min((1 + r_plus_half)/2, 2, 2*r_plus_half))

        u_values_for_step[i] = u[i] + (dt/dx) * (f_minus_half - f_plus_half)
    line.set_ydata(u_values_for_step)

    return line,

animation = FuncAnimation(fig, update, frames=len(t), interval=50, blit=True)
plt.show()


## Q 2.

1D Hydrodynamic solver in Isothermal Conditions

$\circ$  equation of continuity:
   $\frac{\partial \rho}{\partial t} + \frac{\partial (\rho v)}{\partial x} = 0$

$\circ $ Momentum equation:
   $\frac{\partial (\rho v)}{\partial t} + \frac{\partial (\rho v^2)}{\partial x} = -\frac{\partial P}{\partial x}$ ; where P  = $c^{2}_{0} \rho $

let $q_{1} = \rho$, $q_{2} =\rho v$
Then the equation becomes:

----->$\frac{\partial q_{1}}{\partial t} + \frac{\partial (q_{1} v)}{\partial x} = 0$

-----> $\frac{\partial (q_{2})}{\partial t} + \frac{\partial (q_{2} v)}{\partial x} = -\frac{\partial c^{2}_{0} q_{1} }{\partial x}$



In [2]:
def isothermal_shock_tube_animation(nt):
    %matplotlib tk

    # Define parameters
    start, end = -50, 50
    num_points = 100
    points = np.linspace(start, end, num_points)
    delta_x = points[1] - points[0]
    time = 0
    cfl_factor = 0.5
    delta_t = cfl_factor * delta_x / 1.0  # Use isothermal sound speed cs = 1
    
    # Initial conditions
    theta_function = np.piecewise(points, [points <= 0, points > 0], [0, 1])
    density_initial = 2 * theta_function + 1
    velocity_initial = np.zeros_like(points)

    # Plotting initial condition
    fig, ax = plt.subplots()
    line_density = ax.plot(points, density_initial, label="Density")[0]
    ax.set_xlabel(r"$x$")
    ax.set_ylabel(r"Value")
    ax.set_title("Isothermal Shock Tube")

    # Set y-axis limits
    ax.set_ylim(-3, 6)

    # Copying the initial conditions for the array size and later updating them based on the function
    density = density_initial.copy()
    velocity = velocity_initial.copy()

    # Define flux limiter function
    def flux_limiter(r):
        return max(0, min(1, r))

    # Define Algorithm function
    def algorithm(frame):
        nonlocal time, density, velocity

        # Calculate new timestep based on CFL condition
        delta_t = cfl_factor * delta_x / max(np.max(np.abs(velocity)), 1.0)

        # here i do the source term step and half step
        for i in range(num_points):
            delta_m = density[i % num_points] - density[(i-1) % num_points]
            delta_p = density[(i+1) % num_points] - density[i % num_points]
            ratio_m = 0 if delta_m == 0 else (density[(i-1) % num_points] - density[(i-2) % num_points]) / delta_m
            ratio_p = 0 if delta_p == 0 else (density[i % num_points] - density[(i-1) % num_points]) / delta_p

            phi_plus = flux_limiter(ratio_p)
            phi_minus = flux_limiter(ratio_m)

            fp = density[i] * velocity[i] + abs(velocity[i]) * (1 - abs(velocity[i] * delta_t / delta_x)) * phi_plus
            fm = density[(i-1) % num_points] * velocity[(i-1) % num_points] + abs(velocity[(i-1) % num_points]) * (1 - abs(velocity[(i-1) % num_points] * delta_t / delta_x)) * phi_minus

            density[i] += (delta_t / delta_x) * (fm - fp)

        # Source step (pressure force equation)
        for i in range(num_points):
            density[i] += -delta_t * (density[i] * velocity[i])

        # Velocity update
        for i in range(num_points):
            velocity[i] = density[i] / max(1e-6, density[i])  # Avoid division by zero

        # Update the plot with the new solution
        line_density.set_ydata(density)

        time += delta_t
        return line_density

    anim = FuncAnimation(fig, algorithm, frames=nt, interval=50)

    return anim





In [3]:
isothermal_shock_tube_animation(600)